# Building a GDS-Style GraphSAGE Training Pipeline

**A Hands-On Guide to Random Walk Co-occurrence Training**

---

## What You Will Build

This notebook teaches you to build a complete GraphSAGE training pipeline inspired by Neo4j Graph Data Science (GDS). You will implement:

1. **Data Pipeline**: Raw graph → Processed PyG dataset
2. **Feature Extraction**: Computing structural node features
3. **Random Walk Generation**: The theory and implementation
4. **Pair Sampling**: Extracting co-occurrence pairs from walks
5. **Negative Sampling**: Degree-biased sampling strategy
6. **Training Loop**: Skip-gram loss with scale factor

By the end, you'll understand every component of our training pipeline.

---

# Chapter 1: Data Pipeline

## Memory Bank System With Graph Structure

The design of our GraphSAGE model is tightly coupled with the memory bank system from Memory R1. 

A full graph contains 2 types of nodes: **Memory nodes** and **Entity nodes**, and 3 types of edges: **MENTION**, **CAUSED_BY**, **NEXT_EVENT**.

## From Raw Graph to Training-Ready Dataset

### The Data Flow

```
Raw Data (Neo4j, JSON, etc.)
        ↓
Full Graph (Memory + Entity nodes)
        ↓
Memory-Only View (extract subgraph)
        ↓
Feature Extraction (7 dimensions)
        ↓
PyTorch Geometric Dataset
```

### Why Extract Memory-Only View?

Our full graph has:
- **Memory nodes**: The events we want embeddings for
- **Entity nodes**: People, places, things (auxiliary)

For GraphSAGE training, we only need memory nodes. Entity connections are captured in the features.

In [ ]:
# Chapter 1: Building the Data Pipeline

import torch
import numpy as np
from typing import Dict, List, Tuple, Set
from dataclasses import dataclass

# =============================================================================
# Step 1: Define Graph Structure
# =============================================================================

@dataclass
class RawGraph:
    """
    Raw graph data structure.
    
    In production, this comes from Neo4j, JSON files, or databases.
    Here we define the structure for synthetic generation.
    """
    memory_ids: List[str]           # List of memory node IDs
    entity_ids: List[str]           # List of entity node IDs
    caused_by_edges: List[Tuple[str, str]]   # (source, target) - causal
    next_event_edges: List[Tuple[str, str]]  # (source, target) - temporal
    mention_edges: List[Tuple[str, str]]     # (memory, entity) - references

print("Graph structure defined!")
print("\nEdge types in Memory R1:")
print("  1. caused_by:   Memory A caused Memory B")
print("  2. next_event:  Memory A happened before Memory B")
print("  3. mention:     Memory mentions Entity")

In [ ]:
# =============================================================================
# Step 2: Synthetic Graph Generation
# =============================================================================

import random

def generate_synthetic_graph(
    num_memories: int = 100,
    num_entities: int = 20,
    causal_density: float = 0.02,
    temporal_density: float = 0.03,
    mention_density: float = 0.1,
    seed: int = 42
) -> RawGraph:
    """
    Generate a synthetic graph for testing.
    
    Args:
        num_memories: Number of memory nodes
        num_entities: Number of entity nodes
        causal_density: Probability of causal edge between any two memories
        temporal_density: Probability of temporal edge
        mention_density: Probability of memory mentioning entity
        seed: Random seed for reproducibility
    
    Returns:
        RawGraph with all edges
    """
    random.seed(seed)
    np.random.seed(seed)
    
    # Create node IDs
    memory_ids = [f"memory_{i}" for i in range(num_memories)]
    entity_ids = [f"entity_{i}" for i in range(num_entities)]
    
    # Generate causal edges (directed: cause → effect)
    caused_by_edges = []
    for i in range(num_memories):
        for j in range(i + 1, num_memories):  # Only forward edges
            if random.random() < causal_density:
                caused_by_edges.append((memory_ids[i], memory_ids[j]))
    
    # Generate temporal edges (directed: earlier → later)
    next_event_edges = []
    for i in range(num_memories):
        for j in range(i + 1, min(i + 10, num_memories)):  # Local temporal
            if random.random() < temporal_density:
                next_event_edges.append((memory_ids[i], memory_ids[j]))
    
    # Generate mention edges (memory → entity)
    mention_edges = []
    for mem_id in memory_ids:
        for ent_id in entity_ids:
            if random.random() < mention_density:
                mention_edges.append((mem_id, ent_id))
    
    return RawGraph(
        memory_ids=memory_ids,
        entity_ids=entity_ids,
        caused_by_edges=caused_by_edges,
        next_event_edges=next_event_edges,
        mention_edges=mention_edges
    )

# Generate example graph
raw_graph = generate_synthetic_graph(num_memories=100, num_entities=20)

print("Synthetic Graph Generated!")
print("=" * 50)
print(f"Memory nodes:     {len(raw_graph.memory_ids)}")
print(f"Entity nodes:     {len(raw_graph.entity_ids)}")
print(f"Causal edges:     {len(raw_graph.caused_by_edges)}")
print(f"Temporal edges:   {len(raw_graph.next_event_edges)}")
print(f"Mention edges:    {len(raw_graph.mention_edges)}")

In [ ]:
# =============================================================================
# Step 3: Extract Memory-Only View
# =============================================================================

@dataclass
class MemoryGraphView:
    """
    Memory-only subgraph for GraphSAGE training.
    
    This is the 'view' we train on - only memory nodes,
    with edges derived from causal + temporal relationships.
    """
    num_nodes: int
    node_ids: List[str]              # Original memory IDs
    id_to_idx: Dict[str, int]        # Map ID → index
    edge_index: torch.Tensor         # [2, num_edges] in PyG format
    edge_types: List[str]            # Type of each edge


def extract_memory_view(raw_graph: RawGraph, undirected: bool = True) -> MemoryGraphView:
    """
    Extract memory-only subgraph from full graph.
    
    Args:
        raw_graph: Full graph with all node types
        undirected: If True, add reverse edges (recommended for GNN)
    
    Returns:
        MemoryGraphView ready for training
    """
    # Create mapping: memory_id → index
    node_ids = raw_graph.memory_ids
    id_to_idx = {mid: idx for idx, mid in enumerate(node_ids)}
    
    # Collect edges (memory-to-memory only)
    edges = []
    edge_types = []
    
    # Add causal edges
    for src, tgt in raw_graph.caused_by_edges:
        if src in id_to_idx and tgt in id_to_idx:
            edges.append((id_to_idx[src], id_to_idx[tgt]))
            edge_types.append('caused_by')
            if undirected:
                edges.append((id_to_idx[tgt], id_to_idx[src]))
                edge_types.append('caused_by')
    
    # Add temporal edges
    for src, tgt in raw_graph.next_event_edges:
        if src in id_to_idx and tgt in id_to_idx:
            edges.append((id_to_idx[src], id_to_idx[tgt]))
            edge_types.append('next_event')
            if undirected:
                edges.append((id_to_idx[tgt], id_to_idx[src]))
                edge_types.append('next_event')
    
    # Convert to PyG format
    if edges:
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()
    else:
        edge_index = torch.zeros((2, 0), dtype=torch.long)
    
    return MemoryGraphView(
        num_nodes=len(node_ids),
        node_ids=node_ids,
        id_to_idx=id_to_idx,
        edge_index=edge_index,
        edge_types=edge_types
    )

# Extract view
memory_view = extract_memory_view(raw_graph, undirected=True)

print("Memory-Only View Extracted!")
print("=" * 50)
print(f"Nodes:  {memory_view.num_nodes}")
print(f"Edges:  {memory_view.edge_index.shape[1]} (undirected, so 2x original)")
print(f"\nEdge index shape: {memory_view.edge_index.shape}")
print(f"Format: [2, num_edges] where row 0 = source, row 1 = target")
print(f"\nFirst 5 edges:")
for i in range(min(5, memory_view.edge_index.shape[1])):
    src, tgt = memory_view.edge_index[:, i].tolist()
    print(f"  {memory_view.node_ids[src]} → {memory_view.node_ids[tgt]} ({memory_view.edge_types[i]})")

---

# Chapter 2: Feature Extraction

## Computing the 7-Dimensional Node Features

GraphSAGE needs initial node features. We compute 7 structural features:

| Index | Feature | Description | Type |
|-------|---------|-------------|------|
| 0 | `caused_by_in_degree` | Incoming causal edges | Continuous |
| 1 | `next_event_out_degree` | Outgoing temporal edges | Continuous |
| 2 | `num_entities` | Entities mentioned | Continuous |
| 3 | `shared_entity_neighbors` | Neighbors sharing entities | Continuous |
| 4 | `is_cause` | Has outgoing causal edges? | Binary |
| 5 | `is_effect` | Has incoming causal edges? | Binary |
| 6 | `has_successor` | Has temporal successors? | Binary |

### Why These Features?

They capture the **structural role** of each memory:
- Root causes: high `is_cause`, low `is_effect`
- Final effects: low `is_cause`, high `is_effect`
- Hub memories: high degrees

In [ ]:
# =============================================================================
# Feature Extraction Implementation
# =============================================================================

def compute_node_features(
    raw_graph: RawGraph,
    memory_view: MemoryGraphView
) -> torch.Tensor:
    """
    Compute 7-dimensional features for each memory node.
    
    Args:
        raw_graph: Original graph with all edges
        memory_view: Memory-only view with index mapping
    
    Returns:
        features: Tensor of shape [num_nodes, 7]
    """
    num_nodes = memory_view.num_nodes
    id_to_idx = memory_view.id_to_idx
    
    # Initialize feature arrays
    caused_by_in_degree = np.zeros(num_nodes)
    next_event_out_degree = np.zeros(num_nodes)
    num_entities = np.zeros(num_nodes)
    is_cause = np.zeros(num_nodes)
    is_effect = np.zeros(num_nodes)
    has_successor = np.zeros(num_nodes)
    
    # Count causal edges
    for src, tgt in raw_graph.caused_by_edges:
        if src in id_to_idx and tgt in id_to_idx:
            src_idx, tgt_idx = id_to_idx[src], id_to_idx[tgt]
            caused_by_in_degree[tgt_idx] += 1  # Target receives cause
            is_cause[src_idx] = 1              # Source is a cause
            is_effect[tgt_idx] = 1             # Target is an effect
    
    # Count temporal edges
    for src, tgt in raw_graph.next_event_edges:
        if src in id_to_idx and tgt in id_to_idx:
            src_idx = id_to_idx[src]
            next_event_out_degree[src_idx] += 1
            has_successor[src_idx] = 1
    
    # Count entity mentions
    for mem_id, ent_id in raw_graph.mention_edges:
        if mem_id in id_to_idx:
            num_entities[id_to_idx[mem_id]] += 1
    
    # Compute shared entity neighbors (simplified)
    # For each memory, count how many neighbors share at least one entity
    memory_entities = {mid: set() for mid in raw_graph.memory_ids}
    for mem_id, ent_id in raw_graph.mention_edges:
        if mem_id in memory_entities:
            memory_entities[mem_id].add(ent_id)
    
    shared_entity_neighbors = np.zeros(num_nodes)
    # Get neighbors for each node
    neighbors = {i: set() for i in range(num_nodes)}
    for i in range(memory_view.edge_index.shape[1]):
        src, tgt = memory_view.edge_index[:, i].tolist()
        neighbors[src].add(tgt)
    
    for idx in range(num_nodes):
        node_id = memory_view.node_ids[idx]
        node_entities = memory_entities.get(node_id, set())
        if node_entities:
            for neighbor_idx in neighbors[idx]:
                neighbor_id = memory_view.node_ids[neighbor_idx]
                neighbor_entities = memory_entities.get(neighbor_id, set())
                if node_entities & neighbor_entities:  # Intersection
                    shared_entity_neighbors[idx] += 1
    
    # Apply log normalization to continuous features
    # log1p(x) = log(1 + x), handles zeros gracefully
    caused_by_in_degree = np.log1p(caused_by_in_degree)
    next_event_out_degree = np.log1p(next_event_out_degree)
    num_entities = np.log1p(num_entities)
    shared_entity_neighbors = np.log1p(shared_entity_neighbors)
    
    # Stack into feature matrix
    features = np.stack([
        caused_by_in_degree,     # 0
        next_event_out_degree,   # 1
        num_entities,            # 2
        shared_entity_neighbors, # 3
        is_cause,                # 4
        is_effect,               # 5
        has_successor,           # 6
    ], axis=1)
    
    return torch.tensor(features, dtype=torch.float32)

# Compute features
features = compute_node_features(raw_graph, memory_view)

print("Feature Extraction Complete!")
print("=" * 50)
print(f"Feature shape: {features.shape}")
print(f"\nFeature statistics:")
feature_names = [
    'caused_by_in_degree', 'next_event_out_degree', 'num_entities',
    'shared_entity_neighbors', 'is_cause', 'is_effect', 'has_successor'
]
for i, name in enumerate(feature_names):
    col = features[:, i]
    print(f"  {name:25s}: min={col.min():.2f}, max={col.max():.2f}, mean={col.mean():.2f}")

In [ ]:
# Visualize feature distributions
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

for i, name in enumerate(feature_names):
    ax = axes[i]
    ax.hist(features[:, i].numpy(), bins=20, edgecolor='black', alpha=0.7)
    ax.set_title(name)
    ax.set_xlabel('Value')
    ax.set_ylabel('Count')

# Hide empty subplot
axes[7].axis('off')

plt.suptitle('Node Feature Distributions', fontsize=14)
plt.tight_layout()
plt.show()

print("\n💡 Notice:")
print("   - Continuous features (0-3) are log-normalized")
print("   - Binary features (4-6) are 0 or 1")
print("   - This mix helps GraphSAGE learn structural roles")

---

# Chapter 3: Random Walk Theory

## The Core Idea Behind GDS-Style Training

### What is a Random Walk?

A random walk is a sequence of nodes where each step moves to a **random neighbor**:

```
Start at node A
    → randomly pick neighbor B
    → randomly pick neighbor of B → C
    → randomly pick neighbor of C → D
    → ... continue for L steps
    
Result: Walk = [A, B, C, D, ...]
```

### Why Random Walks?

Random walks capture **structural proximity**:
- Nodes appearing together in walks are **structurally close**
- The more often two nodes co-occur, the more similar they are

### The Key Insight (from Node2Vec/DeepWalk)

Treat walks like **sentences** and nodes like **words**:
- Word2Vec: Words appearing in same context are similar
- Node2Vec: Nodes appearing in same walk are similar

This is why we use **skip-gram loss** - same objective as Word2Vec!

In [ ]:
# =============================================================================
# Random Walk Generator Implementation
# =============================================================================

class RandomWalkGenerator:
    """
    Generate random walks on a graph.
    
    This is the foundation of GDS-style training:
    1. Generate many random walks
    2. Extract co-occurring node pairs
    3. Train embeddings with skip-gram loss
    """
    
    def __init__(
        self,
        edge_index: torch.Tensor,
        num_nodes: int,
        walk_length: int = 80,
        walks_per_node: int = 10,
        seed: int = 42
    ):
        """
        Args:
            edge_index: [2, num_edges] tensor
            num_nodes: Total number of nodes
            walk_length: Steps per walk
            walks_per_node: Walks starting from each node
            seed: Random seed
        """
        self.num_nodes = num_nodes
        self.walk_length = walk_length
        self.walks_per_node = walks_per_node
        
        random.seed(seed)
        
        # Build adjacency list for fast neighbor lookup
        self.neighbors = {i: [] for i in range(num_nodes)}
        for i in range(edge_index.shape[1]):
            src, tgt = edge_index[:, i].tolist()
            self.neighbors[src].append(tgt)
        
        # Track nodes with no neighbors (isolated)
        self.isolated_nodes = [i for i in range(num_nodes) if not self.neighbors[i]]
    
    def generate_single_walk(self, start_node: int) -> List[int]:
        """
        Generate one random walk starting from a node.
        
        The walk proceeds by:
        1. Start at start_node
        2. Randomly select a neighbor
        3. Move to that neighbor
        4. Repeat until walk_length reached or stuck
        """
        walk = [start_node]
        current = start_node
        
        for _ in range(self.walk_length - 1):
            neighbors = self.neighbors[current]
            if not neighbors:
                break  # Stuck at node with no outgoing edges
            current = random.choice(neighbors)
            walk.append(current)
        
        return walk
    
    def generate_all_walks(self) -> List[List[int]]:
        """
        Generate walks_per_node walks starting from each node.
        
        Returns:
            List of walks, each walk is a list of node indices
        """
        all_walks = []
        
        for node in range(self.num_nodes):
            if node in self.isolated_nodes:
                continue  # Skip isolated nodes
            
            for _ in range(self.walks_per_node):
                walk = self.generate_single_walk(node)
                if len(walk) > 1:  # Only keep walks with at least 2 nodes
                    all_walks.append(walk)
        
        return all_walks

# Generate walks
walker = RandomWalkGenerator(
    edge_index=memory_view.edge_index,
    num_nodes=memory_view.num_nodes,
    walk_length=20,      # Shorter for demo
    walks_per_node=5,    # Fewer for demo
    seed=42
)

walks = walker.generate_all_walks()

print("Random Walks Generated!")
print("=" * 50)
print(f"Total walks: {len(walks)}")
print(f"Walk length (target): {walker.walk_length}")
print(f"Isolated nodes (skipped): {len(walker.isolated_nodes)}")

# Show example walks
print(f"\nExample walks:")
for i in range(min(3, len(walks))):
    walk = walks[i]
    walk_str = ' → '.join([str(n) for n in walk[:8]])
    if len(walk) > 8:
        walk_str += f' → ... ({len(walk)} total)'
    print(f"  Walk {i+1}: {walk_str}")

In [ ]:
# Visualize walk statistics
import matplotlib.pyplot as plt

walk_lengths = [len(w) for w in walks]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Walk length distribution
axes[0].hist(walk_lengths, bins=20, edgecolor='black', alpha=0.7)
axes[0].axvline(x=np.mean(walk_lengths), color='red', linestyle='--', label=f'Mean: {np.mean(walk_lengths):.1f}')
axes[0].set_xlabel('Walk Length')
axes[0].set_ylabel('Count')
axes[0].set_title('Walk Length Distribution')
axes[0].legend()

# Node visit frequency
node_visits = np.zeros(memory_view.num_nodes)
for walk in walks:
    for node in walk:
        node_visits[node] += 1

axes[1].hist(node_visits, bins=30, edgecolor='black', alpha=0.7)
axes[1].set_xlabel('Times Visited')
axes[1].set_ylabel('Number of Nodes')
axes[1].set_title('Node Visit Frequency')

plt.tight_layout()
plt.show()

print(f"\n💡 Key observations:")
print(f"   - Some walks are shorter (hit dead ends)")
print(f"   - High-degree nodes are visited more often")
print(f"   - This is natural and helps training!")

---

# Chapter 4: Pair Sampling (Co-occurrence Extraction)

## From Walks to Training Pairs

### The Context Window

For each node in a walk, we extract pairs with nodes in a **context window**:

```
Walk: [A, B, C, D, E, F, G]
Window size: 2

For target C:
  Context = [A, B, D, E]  (2 before, 2 after)
  Pairs: (C,A), (C,B), (C,D), (C,E)
```

### Why This Works

Nodes appearing in the same window are **structurally proximate**:
- They're reachable within a few hops
- Random walks naturally capture graph structure
- More walks = better coverage of relationships

In [ ]:
# =============================================================================
# Co-occurrence Pair Sampler Implementation
# =============================================================================

class CooccurrencePairSampler:
    """
    Extract (target, context) pairs from random walks.
    
    This is the Skip-gram sampling strategy:
    - For each node in a walk, pair it with nearby nodes
    - These pairs become positive training examples
    """
    
    def __init__(self, context_window: int = 5):
        """
        Args:
            context_window: How many nodes before/after to consider as context
        """
        self.context_window = context_window
    
    def extract_pairs_from_walk(self, walk: List[int]) -> List[Tuple[int, int]]:
        """
        Extract all (target, context) pairs from a single walk.
        
        For each position i in the walk:
        - target = walk[i]
        - context = walk[i-window : i+window+1], excluding i
        """
        pairs = []
        walk_len = len(walk)
        
        for i, target in enumerate(walk):
            # Define context window boundaries
            start = max(0, i - self.context_window)
            end = min(walk_len, i + self.context_window + 1)
            
            # Extract context nodes (excluding target itself)
            for j in range(start, end):
                if j != i:
                    context = walk[j]
                    pairs.append((target, context))
        
        return pairs
    
    def extract_pairs(self, walks: List[List[int]]) -> List[Tuple[int, int]]:
        """
        Extract pairs from all walks.
        """
        all_pairs = []
        for walk in walks:
            pairs = self.extract_pairs_from_walk(walk)
            all_pairs.extend(pairs)
        return all_pairs

# Extract pairs
pair_sampler = CooccurrencePairSampler(context_window=3)
positive_pairs = pair_sampler.extract_pairs(walks)

print("Positive Pairs Extracted!")
print("=" * 50)
print(f"Total walks: {len(walks)}")
print(f"Context window: {pair_sampler.context_window}")
print(f"Total pairs: {len(positive_pairs):,}")
print(f"\nSignal amplification: {len(positive_pairs) / memory_view.edge_index.shape[1]:.1f}x more pairs than edges!")

# Show example pairs
print(f"\nExample pairs (target, context):")
for i in range(min(5, len(positive_pairs))):
    target, context = positive_pairs[i]
    print(f"  ({target}, {context})")

In [ ]:
# Visualize the pair extraction process
print("Pair Extraction Visualization")
print("=" * 60)

# Take first walk as example
example_walk = walks[0][:10]  # First 10 nodes
window = 2

print(f"\nExample walk: {example_walk}")
print(f"Window size: {window}")
print(f"\n" + "-" * 60)

for i, target in enumerate(example_walk):
    start = max(0, i - window)
    end = min(len(example_walk), i + window + 1)
    
    context_nodes = [example_walk[j] for j in range(start, end) if j != i]
    
    # Visual representation
    visual = []
    for j, node in enumerate(example_walk):
        if j == i:
            visual.append(f"[{node}]")
        elif j >= start and j < end:
            visual.append(f"({node})")
        else:
            visual.append(f" {node} ")
    
    print(f"Position {i}: {' '.join(visual)}")
    print(f"           Target={target}, Context={context_nodes}")
    print(f"           Pairs: {[(target, c) for c in context_nodes]}")
    print()

---

# Chapter 5: Negative Sampling

## Why Negative Samples Are Essential

### The Collapse Problem

If we only train on positive pairs (pull similar nodes together), the model could learn:
- "Make ALL embeddings identical" → Loss = 0!

This is called **embedding collapse**. Negative samples prevent it by:
- Pushing non-co-occurring nodes apart
- Creating contrast between positive and negative

### Degree-Biased Sampling

Not all negatives are equal. We use **degree-biased sampling** from Word2Vec:

```
P(node) ∝ degree(node)^0.75
```

**Why 0.75?**
- Exponent = 1.0: Sample exactly proportional to degree (too biased to hubs)
- Exponent = 0.0: Uniform sampling (ignores structure)
- Exponent = 0.75: Sweet spot (empirically proven by Word2Vec)

In [ ]:
# =============================================================================
# Degree-Biased Negative Sampler Implementation
# =============================================================================

class DegreeBiasedNegativeSampler:
    """
    Sample negative nodes with probability proportional to degree^exponent.
    
    Uses the alias method for O(1) sampling after O(n) preprocessing.
    """
    
    def __init__(
        self,
        edge_index: torch.Tensor,
        num_nodes: int,
        exponent: float = 0.75
    ):
        """
        Args:
            edge_index: Graph edges
            num_nodes: Total nodes
            exponent: Degree smoothing (0.75 is Word2Vec default)
        """
        self.num_nodes = num_nodes
        self.exponent = exponent
        
        # Compute node degrees
        degrees = np.zeros(num_nodes)
        for i in range(edge_index.shape[1]):
            src = edge_index[0, i].item()
            degrees[src] += 1
        
        # Apply exponent and normalize
        # Add small epsilon to avoid zero probability for isolated nodes
        weights = (degrees + 1e-10) ** exponent
        self.probabilities = weights / weights.sum()
        
        # Store degrees for analysis
        self.degrees = degrees
    
    def sample(self, num_samples: int) -> np.ndarray:
        """
        Sample negative node indices.
        
        Args:
            num_samples: Number of negatives to sample
        
        Returns:
            Array of sampled node indices
        """
        return np.random.choice(
            self.num_nodes,
            size=num_samples,
            p=self.probabilities,
            replace=True
        )
    
    def sample_for_pairs(
        self,
        num_pairs: int,
        num_negatives: int = 5
    ) -> np.ndarray:
        """
        Sample negatives for a batch of positive pairs.
        
        Args:
            num_pairs: Number of positive pairs
            num_negatives: Negatives per positive pair
        
        Returns:
            Array of shape [num_pairs, num_negatives]
        """
        total_negatives = num_pairs * num_negatives
        negatives = self.sample(total_negatives)
        return negatives.reshape(num_pairs, num_negatives)

# Create sampler
neg_sampler = DegreeBiasedNegativeSampler(
    edge_index=memory_view.edge_index,
    num_nodes=memory_view.num_nodes,
    exponent=0.75
)

print("Negative Sampler Created!")
print("=" * 50)
print(f"Exponent: {neg_sampler.exponent}")
print(f"Nodes: {neg_sampler.num_nodes}")
print(f"\nDegree statistics:")
print(f"  Min degree: {neg_sampler.degrees.min():.0f}")
print(f"  Max degree: {neg_sampler.degrees.max():.0f}")
print(f"  Mean degree: {neg_sampler.degrees.mean():.2f}")

In [ ]:
# Visualize sampling distribution
import matplotlib.pyplot as plt

# Sample many times to see distribution
samples = neg_sampler.sample(100000)
sample_counts = np.bincount(samples, minlength=neg_sampler.num_nodes)

# Sort nodes by degree for visualization
sorted_indices = np.argsort(neg_sampler.degrees)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Degree distribution
axes[0].bar(range(len(sorted_indices)), neg_sampler.degrees[sorted_indices], alpha=0.7)
axes[0].set_xlabel('Node (sorted by degree)')
axes[0].set_ylabel('Degree')
axes[0].set_title('Node Degrees')

# Sampling probability
axes[1].bar(range(len(sorted_indices)), neg_sampler.probabilities[sorted_indices], alpha=0.7, color='orange')
axes[1].set_xlabel('Node (sorted by degree)')
axes[1].set_ylabel('Sampling Probability')
axes[1].set_title(f'Sampling Probability (exponent={neg_sampler.exponent})')

# Actual samples
axes[2].bar(range(len(sorted_indices)), sample_counts[sorted_indices], alpha=0.7, color='green')
axes[2].set_xlabel('Node (sorted by degree)')
axes[2].set_ylabel('Times Sampled')
axes[2].set_title('Actual Sample Distribution (100K samples)')

plt.tight_layout()
plt.show()

print("\n💡 Key insight:")
print("   - High-degree nodes are sampled more often (harder negatives)")
print("   - But the 0.75 exponent prevents extreme bias")
print("   - This balance is crucial for good training!")

In [ ]:
# Compare different exponents
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

exponents = [0.0, 0.75, 1.0]
titles = ['Uniform (exp=0)', 'Word2Vec (exp=0.75)', 'Proportional (exp=1.0)']

for ax, exp, title in zip(axes, exponents, titles):
    # Compute probabilities for this exponent
    weights = (neg_sampler.degrees + 1e-10) ** exp
    probs = weights / weights.sum()
    
    # Sort by degree
    sorted_probs = probs[sorted_indices]
    
    ax.bar(range(len(sorted_probs)), sorted_probs, alpha=0.7)
    ax.set_xlabel('Node (sorted by degree)')
    ax.set_ylabel('Probability')
    ax.set_title(title)

plt.tight_layout()
plt.show()

print("\n📊 Exponent effects:")
print("   - exp=0.0:  All nodes equally likely (ignores structure)")
print("   - exp=0.75: Balanced - favors high-degree but not extreme")
print("   - exp=1.0:  Proportional to degree (too biased to hubs)")

---

# Chapter 6: The Scale Factor (Critical Discovery)

## The Problem We Discovered

Our first training attempt showed metrics stuck at:
```
pos_prob: 0.71 (should be ~0.95)
neg_prob: 0.50 (should be ~0.10)
```

### Root Cause: Bounded Dot Products

1. Our embeddings are **L2 normalized** (unit vectors)
2. Dot product of unit vectors: **-1 ≤ dot ≤ 1**
3. Sigmoid of bounded input:
   - `sigmoid(1) = 0.73` ← Maximum!
   - `sigmoid(-1) = 0.27` ← Minimum!

**The model CANNOT learn** because probabilities are stuck!

### Solution: Scale Factor

Multiply dot products by a scale factor before sigmoid:

```python
# Before (broken)
loss = -log(sigmoid(dot_product))  # bounded to [0.27, 0.73]

# After (fixed)
loss = -log(sigmoid(dot_product * 5.0))  # can reach [0.007, 0.993]
```

In [ ]:
# Demonstrate the scale factor problem and solution
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np

print("The Scale Factor Problem")
print("=" * 60)

# Dot product range for normalized vectors
dot_products = np.linspace(-1, 1, 100)

# Different scale factors
scale_factors = [1.0, 3.0, 5.0, 10.0]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot sigmoid curves
for scale in scale_factors:
    sigmoid_vals = 1 / (1 + np.exp(-dot_products * scale))
    axes[0].plot(dot_products, sigmoid_vals, label=f'scale={scale}', linewidth=2)

axes[0].axhline(y=0.5, color='gray', linestyle='--', alpha=0.5)
axes[0].axhline(y=0.73, color='red', linestyle=':', alpha=0.5, label='Old ceiling (0.73)')
axes[0].axhline(y=0.27, color='red', linestyle=':', alpha=0.5)
axes[0].set_xlabel('Dot Product')
axes[0].set_ylabel('Sigmoid Output (Probability)')
axes[0].set_title('Effect of Scale Factor on Sigmoid')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Show achievable ranges
scale_values = [1.0, 2.0, 3.0, 5.0, 7.0, 10.0]
max_probs = [1 / (1 + np.exp(-1 * s)) for s in scale_values]
min_probs = [1 / (1 + np.exp(1 * s)) for s in scale_values]

axes[1].fill_between(scale_values, min_probs, max_probs, alpha=0.3, color='blue')
axes[1].plot(scale_values, max_probs, 'b-o', label='Max achievable', linewidth=2)
axes[1].plot(scale_values, min_probs, 'r-o', label='Min achievable', linewidth=2)
axes[1].axhline(y=0.95, color='green', linestyle='--', alpha=0.7, label='Target pos_prob')
axes[1].axhline(y=0.10, color='green', linestyle='--', alpha=0.7, label='Target neg_prob')
axes[1].set_xlabel('Scale Factor')
axes[1].set_ylabel('Probability')
axes[1].set_title('Achievable Probability Range by Scale Factor')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n💡 Key insight:")
print("   scale=1.0: Can only reach [0.27, 0.73] - TRAINING STUCK!")
print("   scale=5.0: Can reach [0.007, 0.993] - TRAINING WORKS!")
print("\n   We use scale_factor=5.0 as default.")

In [ ]:
# =============================================================================
# Skip-gram Loss with Scale Factor
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F

class SkipGramLoss(nn.Module):
    """
    Skip-gram loss with scale factor for normalized embeddings.
    
    Loss = -log(σ(s · pos_score)) - Σ log(σ(-s · neg_score))
    
    Where s is the scale factor.
    """
    
    def __init__(self, scale_factor: float = 5.0):
        super().__init__()
        self.scale_factor = scale_factor
    
    def forward(
        self,
        embeddings: torch.Tensor,
        targets: torch.Tensor,
        contexts: torch.Tensor,
        negatives: torch.Tensor
    ) -> torch.Tensor:
        """
        Compute skip-gram loss.
        
        Args:
            embeddings: All node embeddings [num_nodes, dim]
            targets: Target node indices [batch_size]
            contexts: Context node indices [batch_size]
            negatives: Negative indices [batch_size, num_negatives]
        """
        # Get embeddings
        target_emb = embeddings[targets]      # [batch, dim]
        context_emb = embeddings[contexts]    # [batch, dim]
        negative_emb = embeddings[negatives]  # [batch, num_neg, dim]
        
        # Positive scores: dot product of target and context
        pos_scores = (target_emb * context_emb).sum(dim=1)  # [batch]
        
        # Negative scores: dot product of target and each negative
        neg_scores = torch.bmm(
            negative_emb,
            target_emb.unsqueeze(2)
        ).squeeze(2)  # [batch, num_neg]
        
        # CRITICAL: Apply scale factor!
        pos_scores = pos_scores * self.scale_factor
        neg_scores = neg_scores * self.scale_factor
        
        # Loss: maximize pos similarity, minimize neg similarity
        pos_loss = -F.logsigmoid(pos_scores).mean()
        neg_loss = -F.logsigmoid(-neg_scores).mean()
        
        return pos_loss + neg_loss

# Test the loss
loss_fn = SkipGramLoss(scale_factor=5.0)

# Create dummy data
batch_size = 32
num_nodes = 100
embed_dim = 64
num_negatives = 5

dummy_embeddings = F.normalize(torch.randn(num_nodes, embed_dim), dim=1)
dummy_targets = torch.randint(0, num_nodes, (batch_size,))
dummy_contexts = torch.randint(0, num_nodes, (batch_size,))
dummy_negatives = torch.randint(0, num_nodes, (batch_size, num_negatives))

loss = loss_fn(dummy_embeddings, dummy_targets, dummy_contexts, dummy_negatives)

print("Skip-gram Loss Test")
print("=" * 50)
print(f"Scale factor: {loss_fn.scale_factor}")
print(f"Batch size: {batch_size}")
print(f"Negatives per positive: {num_negatives}")
print(f"Loss value: {loss.item():.4f}")

---

# Chapter 7: Putting It All Together

## The Complete Training Pipeline

Now let's see how all components work together:

```
1. Generate/Load Graph
        ↓
2. Extract Memory View
        ↓
3. Compute Node Features
        ↓
4. Generate Random Walks
        ↓
5. Extract Positive Pairs
        ↓
6. For each training batch:
   a. Sample positive pairs
   b. Sample negative nodes
   c. Compute embeddings (GraphSAGE forward)
   d. Compute skip-gram loss (with scale factor!)
   e. Backpropagate and update
        ↓
7. Evaluate embeddings
```

In [ ]:
# =============================================================================
# Complete Mini Training Example
# =============================================================================

import torch
import torch.nn as nn
import torch.nn.functional as F

# Simple GraphSAGE model
class SimpleGraphSAGE(nn.Module):
    def __init__(self, in_dim, hidden_dim, out_dim):
        super().__init__()
        self.conv1 = nn.Linear(in_dim * 2, hidden_dim)
        self.conv2 = nn.Linear(hidden_dim * 2, out_dim)
        self.dropout = nn.Dropout(0.3)
    
    def aggregate(self, x, edge_index, num_nodes):
        """Mean aggregation of neighbors."""
        agg = torch.zeros(num_nodes, x.size(1))
        count = torch.zeros(num_nodes, 1)
        
        src, tgt = edge_index
        agg.index_add_(0, tgt, x[src])
        count.index_add_(0, tgt, torch.ones(src.size(0), 1))
        
        return agg / count.clamp(min=1)
    
    def forward(self, x, edge_index):
        num_nodes = x.size(0)
        
        # Layer 1
        neigh = self.aggregate(x, edge_index, num_nodes)
        x = torch.cat([x, neigh], dim=1)
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        
        # Layer 2
        neigh = self.aggregate(x, edge_index, num_nodes)
        x = torch.cat([x, neigh], dim=1)
        x = self.conv2(x)
        
        # L2 normalize
        x = F.normalize(x, p=2, dim=1)
        
        return x

# Initialize model
model = SimpleGraphSAGE(in_dim=7, hidden_dim=32, out_dim=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = SkipGramLoss(scale_factor=5.0)

print("Mini Training Pipeline")
print("=" * 60)

# Training loop
num_epochs = 10
batch_size = 64
num_negatives = 5

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    num_batches = 0
    
    # Shuffle pairs
    indices = np.random.permutation(len(positive_pairs))
    
    for i in range(0, len(indices), batch_size):
        batch_indices = indices[i:i+batch_size]
        if len(batch_indices) < batch_size // 2:
            continue
        
        # Get batch pairs
        batch_pairs = [positive_pairs[j] for j in batch_indices]
        targets = torch.tensor([p[0] for p in batch_pairs])
        contexts = torch.tensor([p[1] for p in batch_pairs])
        
        # Sample negatives
        negatives = torch.tensor(
            neg_sampler.sample_for_pairs(len(batch_pairs), num_negatives)
        )
        
        # Forward pass
        optimizer.zero_grad()
        embeddings = model(features, memory_view.edge_index)
        
        # Compute loss
        loss = loss_fn(embeddings, targets, contexts, negatives)
        
        # Backward pass
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    avg_loss = total_loss / num_batches
    print(f"Epoch {epoch+1:2d} | Loss: {avg_loss:.4f}")

print("\n✅ Training complete!")

In [ ]:
# Evaluate the embeddings
model.eval()
with torch.no_grad():
    final_embeddings = model(features, memory_view.edge_index)

print("Embedding Quality Check")
print("=" * 60)

# Check normalization
norms = final_embeddings.norm(dim=1)
print(f"\n1. Normalization check:")
print(f"   All norms ≈ 1.0: {torch.allclose(norms, torch.ones_like(norms), atol=1e-5)}")

# Check neighbor similarity vs random similarity
edge_index = memory_view.edge_index

# Neighbor similarities
src_emb = final_embeddings[edge_index[0]]
tgt_emb = final_embeddings[edge_index[1]]
neighbor_sim = (src_emb * tgt_emb).sum(dim=1).mean().item()

# Random similarities
random_idx1 = torch.randint(0, final_embeddings.size(0), (1000,))
random_idx2 = torch.randint(0, final_embeddings.size(0), (1000,))
random_sim = (final_embeddings[random_idx1] * final_embeddings[random_idx2]).sum(dim=1).mean().item()

print(f"\n2. Similarity check:")
print(f"   Neighbor similarity: {neighbor_sim:.4f}")
print(f"   Random similarity:   {random_sim:.4f}")
print(f"   Gap:                 {neighbor_sim - random_sim:.4f}")

if neighbor_sim > random_sim + 0.05:
    print("\n✅ Embeddings capture graph structure!")
else:
    print("\n⚠️  Gap is small - may need more training.")

---

# Summary: What You've Learned

## The GDS-Style Training Pipeline

### 1. Data Pipeline
```
Raw Graph → Memory View → Features → PyG Dataset
```

### 2. Feature Extraction
- 7-dimensional structural features
- Continuous (log-normalized) + Binary features
- Captures node roles in the graph

### 3. Random Walk Generation
- Start from each node
- Randomly traverse neighbors
- Captures structural proximity

### 4. Pair Sampling
- Context window extracts co-occurring pairs
- Amplifies training signal (edges → millions of pairs)

### 5. Negative Sampling
- Degree-biased: `P(node) ∝ degree^0.75`
- Prevents embedding collapse

### 6. Scale Factor (Critical!)
- L2 normalized embeddings → bounded dot products
- Without scale: sigmoid stuck at [0.27, 0.73]
- With scale=5.0: sigmoid can reach [0.007, 0.993]

## Key Takeaways

1. **Random walks** convert graph structure into sequences
2. **Co-occurrence** creates massive training signal
3. **Skip-gram loss** learns from similarity/dissimilarity
4. **Scale factor is essential** for normalized embeddings
5. **Evaluation metrics > training loss** for quality assessment

---

# Exercises

## Exercise 1: Walk Parameters
Experiment with different `walk_length` (40, 80, 160) and `walks_per_node` (5, 10, 20). How do they affect:
- Number of training pairs?
- Training time?
- Final embedding quality?

## Exercise 2: Context Window
Try context windows of 2, 5, 10, and 20. What happens to the training signal and embedding quality?

## Exercise 3: Scale Factor
Train with scale factors 1.0, 3.0, 5.0, and 10.0. Plot the `pos_prob` and `neg_prob` curves.

## Exercise 4: Negative Sampling Exponent
Compare exponents 0.0, 0.5, 0.75, and 1.0. Which gives the best link prediction AUC?

## Exercise 5: Feature Ablation
Train with subsets of features (e.g., only degrees, only binary). Which features are most important?

---

**Congratulations!** You now understand how to build a GDS-style GraphSAGE training pipeline from scratch.